# Ejercicios Spark

## Pasos Generales para todos los programas

los pasos generales son:
* las importaciones
* la creacion de SparkSession y SparkContext

### Importaciones necesarias

In [1]:
import findspark
findspark.init()

import pandas as pd
import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import regexp_replace, col


### creacion de SparkSession y SparkContext

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [3]:
def espacio():
    print("\n" + "-" * 50 + "\n")

## Ejercicio 1

* Genera un RDD con la siguiente lista [23,47,6,118,9,2,23,42,64,32,76,3,8,23,12,434,7534,24,9,23]
* Filtra por los elementos únicos del RDD
* Multiplica por 6 todos los elementos del RDD anterior
* Filtra el elemento resultante por los elementos pares

In [3]:
# Lista original
lista_numeros =  [23,47,6,118,9,2,23,42,64,32,76,3,8,23,12,434,7534,24,9,23]

# RDD con los numeros
numeros_rdd = sc.parallelize(lista_numeros)

# Filtrar por los elementos unicos 
numeros_unicos_rdd = numeros_rdd.distinct()

# Multiplica por 6 todos los elementos del RDD
numero_multiplicados_rdd = numeros_unicos_rdd.map(lambda x: x*6)

# Filtrar los elementos pares
numeros_pares_rdd = numero_multiplicados_rdd.filter(lambda x: x%2==0)

Muestra de los resultados

In [4]:
# Lista original
print("Lista Original: {}".format(numeros_rdd.collect()))

# Lista con los elementos unicos
print("Lista numeros unicos: {}".format(numeros_unicos_rdd.collect()))

# Lista con los numeros multiplicados
print("Lista numeros unicos multiplicados: {}".format(numero_multiplicados_rdd.collect()))

# Lista numeros pares
print("Lista numeros unicos multiplicados y pares: {}".format(numeros_pares_rdd.collect()))

Lista Original: [23, 47, 6, 118, 9, 2, 23, 42, 64, 32, 76, 3, 8, 23, 12, 434, 7534, 24, 9, 23]
Lista numeros unicos: [64, 32, 8, 24, 9, 2, 42, 434, 3, 76, 12, 6, 118, 7534, 23, 47]
Lista numeros unicos multiplicados: [384, 192, 48, 144, 54, 12, 252, 2604, 18, 456, 72, 36, 708, 45204, 138, 282]
Lista numeros unicos multiplicados y pares: [384, 192, 48, 144, 54, 12, 252, 2604, 18, 456, 72, 36, 708, 45204, 138, 282]


## Ejercicio 2

* Genera un RDD con la siguiente lista [(‘Alan’,10),(´Beto´,13),(‘Carlos’,20),(‘Daniel’,25), (‘Alan’,15),(´Marlen´,13),(‘Pedro’,20),(‘Daniel’,10), (‘Pablo’,10),(´Juan´,13),(‘Cassy’,20),(‘Diana’,25)]
* Obtén la suma de las edades agrupadas por el key
* Ordena los RDDs con base al resultado
* Cuenta cuantas veces aparece cada valor 
* Cuenta cuantas veces aparece cada key


In [22]:
# Generar un RDD con información
data_personas =  sc.parallelize([('Alan',10),('Beto',13),('Carlos',20),('Daniel',25), ('Alan',15),('Marlen',13),('Pedro',20),('Daniel',10), ('Pablo',10),('Juan',13),('Cassy',20),('Diana',25)])

# Obtener suma de las edades agrupadas por key
personas_suma = data_personas.reduceByKey(lambda x, y: x+y)

# ordenar los RDDs en base al resultado
personas_suma_ordenada = personas_suma.sortBy(lambda x: x[1])

# Cuenta cuantas veces aparece cada valor y cada key
conteo_keys = data_personas.countByKey()
conteo_values = data_personas.map(lambda x: x[1]).countByValue()

Muestra de resultados y usaremos una funcion auxiliar para imprimir unos datos

In [54]:
def printDict(data: dict, title: str) -> None:
    if not isinstance(data, dict):
        print("Error: la data debe ser un diccionario.")
        return
    if not isinstance(title, str):
        print("Error: el titulo debe ser un diccionario.")
        return 
    
    print(title + "\n")
    
    for key, value in data.items():
        print("Key: {}, Cantidad: {}".format(key, value))

In [55]:
# Obtener la suma de las edades agrupadas por el key
print("Suma edades por key: {} ".format(personas_suma.collect()))

espacio()

# Ordenar los RDDs en base al resultado
print("Suma edades por key: {} ".format(personas_suma_ordenada.collect()))

espacio()

# Numero de values y keys iguales

printDict(conteo_keys, "cantidad de keys")

espacio()

printDict(conteo_values, "Cantidad de Values")


Suma edades por key: [('Carlos', 20), ('Juan', 13), ('Beto', 13), ('Pablo', 10), ('Diana', 25), ('Marlen', 13), ('Pedro', 20), ('Alan', 25), ('Daniel', 35), ('Cassy', 20)] 

--------------------------------------------------

Suma edades por key: [('Pablo', 10), ('Juan', 13), ('Beto', 13), ('Marlen', 13), ('Carlos', 20), ('Pedro', 20), ('Cassy', 20), ('Diana', 25), ('Alan', 25), ('Daniel', 35)] 

--------------------------------------------------

cantidad de keys

Key: Alan, Cantidad: 2
Key: Beto, Cantidad: 1
Key: Carlos, Cantidad: 1
Key: Daniel, Cantidad: 2
Key: Marlen, Cantidad: 1
Key: Pedro, Cantidad: 1
Key: Pablo, Cantidad: 1
Key: Juan, Cantidad: 1
Key: Cassy, Cantidad: 1
Key: Diana, Cantidad: 1

--------------------------------------------------

Cantidad de Values

Key: 10, Cantidad: 3
Key: 13, Cantidad: 3
Key: 20, Cantidad: 3
Key: 25, Cantidad: 2
Key: 15, Cantidad: 1


## Ejercicio 3

* Inicializa una sesión de spark en consola y coloca en una variable llamada playersInfo la información del archivo t_players_info.csv y muestra el resultado en pantalla
* Muestra el tipo de dato de cada columna
* Ordena los valores de menor a mayor de acuerdo a la columna Rank y muestra el resultado en pantalla
* Devuelve el conteo de todos los jugadores y muestra el resultado en pantalla
* Obtén un conteo de jugadores de acuerdo a su deporte y muestra el resultado en pantalla
* Obtén un conteo de jugadores de acuerdo a su género y muestra el resultado en pantalla
* Devuelve el resultado de todos los jugadores cuyo nombre comience con la letra “A”


In [100]:
# cargar la informacion del archivo t_players_info.csv
playersInfo = spark.read.csv("data/t_players_info.csv", sep=",", header=True, inferSchema=True)
playersInfo.show(10)

+----+-----------------+----------+--------------------+------+
|Rank|             Name|     Sport|      Wikipedia Page|Gender|
+----+-----------------+----------+--------------------+------+
|  55|    Aaron Rodgers|  Football|http://en.wikiped...|  Male|
|  95|       Adam Scott|      Golf|https://en.wikipe...|  Male|
|  60|  Adrian Gonzalez|  Baseball|http://en.wikiped...|  Male|
|  48|   Alex Rodriguez|  Baseball|http://en.wikiped...|  Male|
|  93|  Alfonso Soriano|  Baseball|http://en.wikiped...|  Male|
|  27|Amar'e Stoudemire|Basketball|http://en.wikiped...|  Male|
|  75|       Barry Zito|  Baseball|http://en.wikiped...|  Male|
|  47|    Blake Griffin|Basketball|http://en.wikiped...|  Male|
|  91|   Branden Albert|  Football|http://en.wikiped...|  Male|
|  97| Brandon Marshall|  Football|http://en.wikiped...|  Male|
+----+-----------------+----------+--------------------+------+
only showing top 10 rows



In [101]:
# Muestra el tipo de dato de cada columna
playersInfo.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Wikipedia Page: string (nullable = true)
 |-- Gender: string (nullable = true)



In [102]:
# ordena los valores de menor a mayor de acuerdo con la columna Rank
playersInfo = df.sort("Rank")
playersInfo.show(5)

+----+-----------------+----------+--------------------+------+
|Rank|             Name|     Sport|      Wikipedia Page|Gender|
+----+-----------------+----------+--------------------+------+
|   1| Floyd Mayweather|    Boxing|http://en.wikiped...|  Male|
|   2|Cristiano Ronaldo|    Soccer|http://en.wikiped...|  Male|
|   3|     LeBron James|Basketball|http://en.wikiped...|  Male|
|   4|     Lionel Messi|    Soccer|http://en.wikiped...|  Male|
|   5|      Kobe Bryant|Basketball|http://en.wikiped...|  Male|
+----+-----------------+----------+--------------------+------+
only showing top 5 rows



In [103]:
# Devuelve el conteo de todos los jugadores y muestra el resultado en pantalla
playersInfo.count()

100

In [104]:
# Obtén un conteo de jugadores de acuerdo a su deporte y muestra el resultado en pantalla
jugadores_por_deporte = playersInfo.groupBy("Sport").agg(count("Sport").alias("cantidad"))
jugadores_por_deporte.show()

+----------+--------+
|     Sport|cantidad|
+----------+--------+
|    Tennis|       6|
|    Boxing|       4|
|     Track|       1|
|      Golf|       5|
|Basketball|      18|
|    Racing|       6|
|  Baseball|      27|
|   Cricket|       1|
|    Soccer|      15|
|  Football|      17|
+----------+--------+



In [105]:
# Obtén un conteo de jugadores de acuerdo a su género y muestra el resultado en pantalla
jugadores_por_genero = playersInfo.groupBy("Gender").agg(count("Gender").alias("Cantidad"))
jugadores_por_genero.show()

+------+--------+
|Gender|Cantidad|
+------+--------+
|Female|       3|
|  Male|      97|
+------+--------+



In [106]:
# Devuelve el resultado de todos los jugadores cuyo nombre comience con la letra “A”
df_con_A = playersInfo.filter(df.Name.startswith("A") )
df_con_A.show()

+----+-----------------+----------+--------------------+------+
|Rank|             Name|     Sport|      Wikipedia Page|Gender|
+----+-----------------+----------+--------------------+------+
|  27|Amar'e Stoudemire|Basketball|http://en.wikiped...|  Male|
|  48|   Alex Rodriguez|  Baseball|http://en.wikiped...|  Male|
|  55|    Aaron Rodgers|  Football|http://en.wikiped...|  Male|
|  60|  Adrian Gonzalez|  Baseball|http://en.wikiped...|  Male|
|  77|    Albert Pujols|  Baseball|http://en.wikiped...|  Male|
|  93|  Alfonso Soriano|  Baseball|http://en.wikiped...|  Male|
|  95|       Adam Scott|      Golf|https://en.wikipe...|  Male|
+----+-----------------+----------+--------------------+------+



## Ejercicio 4

* Inicializa una sesión de spark en consola y coloca en una variable llamada playersSalary la información del archivo t_players_salary.csv y muestra el resultado en pantalla
* Muestra el tipo de dato de cada columna
* Ordena los valores de mayor a menor de acuerdo a la columna Total Pay y muestra el resultado en pantalla 
* Devuelve el conteo de todos los salarios por juego ganado mayores a \$15,300,000 y muestra el resultado en pantalla
* Devuelve el conteo de todos los salarios por endoso mayores a $800,000 y muestra el resultado en pantalla
* Devuelve el Salario total máximo de todo el set de datos y muestra el resultado en pantalla
* Devuelve el Salario total mínimo de todo el set de datos y muestra el resultado en pantalla
* Devuelve la suma y promedio de los salarios de cada columna y muestra el resultado en pantalla


### Inicializa una sesión de spark en consola y coloca en una variable llamada playersSalary la información del archivo t_players_salary.csv y muestra el resultado en pantalla 

In [34]:
# cargar la informacion del archivo  t_players_salary.csv
playersSalary = spark.read.csv("data/t_players_salary.csv", sep="|", header=True, inferSchema=True)
playersSalary.show(10)

+----+-----------+---------------+------------+
|Rank|  Total Pay|Salary/Winnings|Endorsements|
+----+-----------+---------------+------------+
|  55|$22,000,000|    $14,500,000|  $7,500,000|
|  95|$17,700,000|     $8,700,000|  $9,000,000|
|  60|$21,500,000|    $21,100,000|    $400,000|
|  48|$22,900,000|    $22,600,000|    $300,000|
|  93|$18,050,000|    $18,000,000|     $50,000|
|  27|$26,700,000|    $21,700,000|  $5,000,000|
|  75|$20,400,000|    $20,300,000|    $100,000|
|  47|$23,000,000|    $16,500,000|  $6,500,000|
|  91|$18,435,000|    $18,400,000|     $35,000|
|  97|$17,500,000|    $16,800,000|    $700,000|
+----+-----------+---------------+------------+
only showing top 10 rows



### Muestra el tipo de dato de cada columna

In [47]:
# Muestra el tipo de dato de cada columna
playersSalary.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Total Pay: string (nullable = true)
 |-- Salary/Winnings: string (nullable = true)
 |-- Endorsements: string (nullable = true)



### Ordena los valores de mayor a menor de acuerdo a la columna Total Pay y muestra el resultado en pantalla

Como los valores son de tipo String se nos deficultara trabajar con numeros por algunos errores que podrian dar como se podra ver en el siguiente ejemplo:

In [111]:
ps_ordenado = playersSalary.sort("Total Pay")
ps_ordenado.show(10)

+----+------------+---------------+------------+
|Rank|   Total Pay|Salary/Winnings|Endorsements|
+----+------------+---------------+------------+
|   1|$105,000,000|   $105,000,000|          $0|
| 100| $17,300,000|    $13,300,000|  $4,000,000|
|  99| $17,400,000|    $17,200,000|    $200,000|
|  97| $17,500,000|    $16,800,000|    $700,000|
|  97| $17,500,000|    $17,300,000|    $200,000|
|  96| $17,600,000|    $17,400,000|    $200,000|
|  95| $17,700,000|     $8,700,000|  $9,000,000|
|  94| $18,000,000|    $17,900,000|    $100,000|
|  93| $18,050,000|    $18,000,000|     $50,000|
|  92| $18,300,000|    $16,800,000|  $1,500,000|
+----+------------+---------------+------------+
only showing top 10 rows



Para evitar estos problemas usaremos la siguiente funcion de forma auxiliar:

In [175]:
# Funcion para regresar un df con el formato de una columna bien

def formatDF(dataframe, campo):
    new_field = "{} Numeric".format(campo)
    # Eliminar los caracteres no numéricos de la columna 'Total Pay'
    df_numbers = dataframe.withColumn(new_field, regexp_replace(col(campo), '[^\d]', '').cast('double')).drop(campo)

    # Dejamos los numeros con un buen formato
    return df_numbers.select("*", format_number(new_field, 2).alias(campo))

Ordena los valores de mayor a menor de acuerdo a la columna Total Pay y muestra el resultado en pantalla **SIN SQL**


In [176]:
df_bien = formatDF(playersSalary, "Total Pay")
df_bien.sort(desc("Total Pay Numeric")).show(5)

+----+---------------+------------+-----------------+--------------+
|Rank|Salary/Winnings|Endorsements|Total Pay Numeric|     Total Pay|
+----+---------------+------------+-----------------+--------------+
|   1|   $105,000,000|          $0|           1.05E8|105,000,000.00|
|   2|    $52,000,000| $28,000,000|            8.0E7| 80,000,000.00|
|   3|    $19,300,000| $53,000,000|           7.23E7| 72,300,000.00|
|   4|    $41,700,000| $23,000,000|           6.47E7| 64,700,000.00|
|   5|    $30,500,000| $31,000,000|           6.15E7| 61,500,000.00|
+----+---------------+------------+-----------------+--------------+
only showing top 5 rows



Ordena los valores de mayor a menor de acuerdo a la columna Total Pay y muestra el resultado en pantalla **CON SQL**

In [177]:
# DataFrame como un Temporary Table
playersSalary.createOrReplaceTempView("temp_table")

# Ejecucion del SQL

query = """
    SELECT * 
    FROM temp_table
    ORDER BY CAST(REPLACE(REPLACE(`Total Pay`, '$', ''), ',', '') AS DOUBLE) DESC
"""

spark.sql(query).show(10)


+----+------------+---------------+------------+
|Rank|   Total Pay|Salary/Winnings|Endorsements|
+----+------------+---------------+------------+
|   1|$105,000,000|   $105,000,000|          $0|
|   2| $80,000,000|    $52,000,000| $28,000,000|
|   3| $72,300,000|    $19,300,000| $53,000,000|
|   4| $64,700,000|    $41,700,000| $23,000,000|
|   5| $61,500,000|    $30,500,000| $31,000,000|
|   6| $61,200,000|     $6,200,000| $55,000,000|
|   7| $56,200,000|     $4,200,000| $52,000,000|
|   8| $53,200,000|     $5,200,000| $48,000,000|
|   9| $44,500,000|    $14,500,000| $30,000,000|
|  10| $43,800,000|    $42,000,000|  $1,800,000|
+----+------------+---------------+------------+
only showing top 10 rows



###  Devuelve el conteo de todos los salarios por juego ganado mayores a $15,300,000 y muestra el resultado en pantalla 
**SIN SQL**

In [178]:
df_bien = formatDF(playersSalary, "Salary/Winnings")
df_bien.filter(df_bien["Salary/Winnings Numeric"] > 15_300_000).count()

76

**CON SQL**

In [179]:
# Devuelve el conteo de todos los salarios por juego ganado mayores a $15,300,000 y muestra el resultado en pantalla

query_cantida_de_juegos = """
    SELECT COUNT(*) as `cantidad`, 
    FORMAT_NUMBER(AVG(CAST(REPLACE(REPLACE(`Salary/Winnings`, '$', ''), ',', '') AS DOUBLE)),2) AS `promedio`,
    FORMAT_NUMBER(SUM(CAST(REPLACE(REPLACE(`Salary/Winnings`, '$', ''), ',', '') AS DOUBLE)),2) AS `sumatoria`
    FROM temp_table
    WHERE CAST(REPLACE(REPLACE(`Salary/Winnings`, '$', ''), ',', '') AS INT) > 15300000;
"""

spark.sql(query_cantida_de_juegos).show(10)


+--------+-------------+----------------+
|cantidad|     promedio|       sumatoria|
+--------+-------------+----------------+
|      76|23,236,842.11|1,766,000,000.00|
+--------+-------------+----------------+



### Devuelve el conteo de todos los salarios por endoso mayores a \$800,000 y muestra el resultado en pantalla 
**SIN SQL**


In [180]:
df_bien = formatDF(playersSalary, "Endorsements")
df_bien.filter(df_bien["Endorsements Numeric"] > 800_000).count()

65

**CONS SQL**


In [181]:
# Devuelve el conteo de todos los salarios por endoso mayores a $800,000 y muestra el resultado en pantalla
query_endorsements_mayores = """
    SELECT COUNT(*) as `cantidad`, 
    FORMAT_NUMBER(AVG(CAST(REPLACE(REPLACE(`Endorsements`, '$', ''), ',', '') AS DOUBLE)),2) AS `promedio`,
    FORMAT_NUMBER(SUM(CAST(REPLACE(REPLACE(`Endorsements`, '$', ''), ',', '') AS DOUBLE)),2) AS `sumatoria`
    FROM temp_table
    WHERE CAST(REPLACE(REPLACE(`Endorsements`, '$', ''), ',', '') AS INT) > 800000;
"""

spark.sql(query_endorsements_mayores).show(10)

+--------+-------------+--------------+
|cantidad|     promedio|     sumatoria|
+--------+-------------+--------------+
|      65|11,238,461.54|730,500,000.00|
+--------+-------------+--------------+



### Devuelve el Salario total máximo de todo el set de datos y muestra el resultado en pantalla
**Sin SQL**

In [183]:
df_bien = formatDF(playersSalary, "Total Pay")
resultado = df_bien.orderBy(desc("Total Pay Numeric")).first() # Row Type
resultado = spark.createDataFrame([resultado]) # DF type
resultado.show()

+----+---------------+------------+-----------------+--------------+
|Rank|Salary/Winnings|Endorsements|Total Pay Numeric|     Total Pay|
+----+---------------+------------+-----------------+--------------+
|   1|   $105,000,000|          $0|           1.05E8|105,000,000.00|
+----+---------------+------------+-----------------+--------------+



**CON SQL**

In [184]:
query = """
    SELECT * 
    FROM temp_table
    ORDER BY CAST(REPLACE(REPLACE(`Total Pay`, '$', ''), ',', '') AS DOUBLE) DESC 
    LIMIT 1
"""

spark.sql(query).show()


+----+------------+---------------+------------+
|Rank|   Total Pay|Salary/Winnings|Endorsements|
+----+------------+---------------+------------+
|   1|$105,000,000|   $105,000,000|          $0|
+----+------------+---------------+------------+



### Devuelve el Salario total mínimo de todo el set de datos y muestra el resultado en pantalla
**SIN SQL**

In [185]:
df_bien = formatDF(playersSalary, "Total Pay")
resultado = df_bien.orderBy("Total Pay Numeric").first() # Row Type
resultado = spark.createDataFrame([resultado]) # DF type
resultado.show()

+----+---------------+------------+-----------------+-------------+
|Rank|Salary/Winnings|Endorsements|Total Pay Numeric|    Total Pay|
+----+---------------+------------+-----------------+-------------+
| 100|    $13,300,000|  $4,000,000|           1.73E7|17,300,000.00|
+----+---------------+------------+-----------------+-------------+



**CON SQL**

In [186]:
query = """
    SELECT * 
    FROM temp_table
    ORDER BY CAST(REPLACE(REPLACE(`Total Pay`, '$', ''), ',', '') AS DOUBLE) 
    LIMIT 1
"""

spark.sql(query).show()

+----+-----------+---------------+------------+
|Rank|  Total Pay|Salary/Winnings|Endorsements|
+----+-----------+---------------+------------+
| 100|$17,300,000|    $13,300,000|  $4,000,000|
+----+-----------+---------------+------------+



### Devuelve la suma y promedio de los salarios de cada columna y muestra el resultado en pantalla
**SIN SQL**

In [187]:
# Funcion aux simplemente para ahorrar codigo
def calColum(funcion, campo):
    return funcion(campo).alias(funcion.__name__+ " " + campo.split(" Numeric")[0])

In [189]:
df_bien = formatDF(playersSalary, "Total Pay")
df_bien = formatDF(df_bien, "Salary/Winnings")
df_bien = formatDF(df_bien, "Endorsements")
df_bien.select(
    calColum(sum, "Total Pay Numeric"), 
    calColum(avg, "Total Pay Numeric"),
    calColum(sum, "Salary/Winnings Numeric"), 
    calColum(avg, "Salary/Winnings Numeric"),
    calColum(sum, "Endorsements Numeric"), 
    calColum(avg, "Endorsements Numeric")
).show()

+-------------+-------------+-------------------+-------------------+----------------+----------------+
|sum Total Pay|avg Total Pay|sum Salary/Winnings|avg Salary/Winnings|sum Endorsements|avg Endorsements|
+-------------+-------------+-------------------+-------------------+----------------+----------------+
|   2.751385E9|   2.751385E7|           2.0099E9|           2.0099E7|       7.41485E8|       7414850.0|
+-------------+-------------+-------------------+-------------------+----------------+----------------+



**CON SQL**

In [192]:
# Devuelve el conteo de todos los salarios por endoso mayores a $800,000 y muestra el resultado en pantalla
query_endorsements_mayores = """
    SELECT COUNT(*) as `cantidad`, 
    FORMAT_NUMBER(AVG(CAST(REPLACE(REPLACE(`Total Pay`, '$', ''), ',', '') AS DOUBLE)),2) AS `avg Total Pay`,
    FORMAT_NUMBER(SUM(CAST(REPLACE(REPLACE(`Total Pay`, '$', ''), ',', '') AS DOUBLE)),2) AS `sum Total Pay`,
    FORMAT_NUMBER(AVG(CAST(REPLACE(REPLACE(`Salary/Winnings`, '$', ''), ',', '') AS DOUBLE)),2) AS `avg Salary/Winnings`,
    FORMAT_NUMBER(SUM(CAST(REPLACE(REPLACE(`Salary/Winnings`, '$', ''), ',', '') AS DOUBLE)),2) AS `sum Salary/Winnings`,
    FORMAT_NUMBER(AVG(CAST(REPLACE(REPLACE(`Endorsements`, '$', ''), ',', '') AS DOUBLE)),2) AS `avg Endorsements`,
    FORMAT_NUMBER(SUM(CAST(REPLACE(REPLACE(`Endorsements`, '$', ''), ',', '') AS DOUBLE)),2) AS `sum Endorsements`
    FROM temp_table
"""

spark.sql(query_endorsements_mayores).show(10)

+--------+-------------+----------------+-------------------+-------------------+----------------+----------------+
|cantidad|avg Total Pay|   sum Total Pay|avg Salary/Winnings|sum Salary/Winnings|avg Endorsements|sum Endorsements|
+--------+-------------+----------------+-------------------+-------------------+----------------+----------------+
|     100|27,513,850.00|2,751,385,000.00|      20,099,000.00|   2,009,900,000.00|    7,414,850.00|  741,485,000.00|
+--------+-------------+----------------+-------------------+-------------------+----------------+----------------+



#### EJemplos de los comandos en consola

In [ ]:
# 1 cargar datos
val playersSalary = spark.read.option("header", "true").option("inferSchema", "true").option("sep","|").csv("C:\\Users\\SoporteTI\\Documents\\ejemplos\\Data\\t_players_salary.csv")
# mostrarlos
playrsSalary.show()
# 2 mostrar los tipos de cada columna
playersSalary.printSchema
# 3 ordenar
playersSalary.createOrReplaceTempView("temp_table")
val query = """
    SELECT * 
    FROM temp_table
    ORDER BY CAST(REPLACE(REPLACE(`Total Pay`, '$', ''), ',', '') AS INT)
"""

spark.sql(query).show(10)

# Devuelte son consultas asi que pues ahi tan los querys arriba

### Ejercicio 5

* Inicializa una sesión de spark en consola y coloca en una variable llamada playersPersInfo la información del archivo t_players_info.csv, t_players_salary.csv, t_players_pers_info.csv considerando la realización de un left join; usa como llave primaria el campo Rank y muestra el resultado en pantalla.
* Muestra el tipo de dato de cada columna.
* Filtra solo por los jugadores que no viven en los Estados Unidos
* Ordena los jugadores del más joven al más veterano 
* Escribe el resultado en un nuevo CSV delimitado por pipes “|” y nómbralo csv_result_players_info.csv

### coloca en una variable llamada playersPersInfo la información del archivo t_players_info.csv, t_players_salary.csv, t_players_pers_info.csv considerando la realización de un left join; usa como llave primaria el campo Rank y muestra el resultado en pantalla.

In [46]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

schema = StructType([
    StructField("Rank", IntegerType(), True),
    StructField("Country", StringType(), True),
    StructField("birthday year", IntegerType(), True),
    StructField("Date of Birth", DateType(), True),
    StructField("Region", StringType(), True),
    StructField("goles", IntegerType(), True),
])

# Cargamos toda la informacion
players_info = spark.read.csv("data/t_players_info.csv", sep=",", header=True, inferSchema=True)
players_salary = spark.read.csv("data/t_players_salary.csv", sep="|", header=True, inferSchema=True)
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
players_pers = spark.read.csv("data/t_players_pers_info.csv", sep=",", header=False, inferSchema=False, schema=schema, dateFormat="dd/MM/yyyy")

# realizamos el left Join para unir la informacion en un solo datagrame
playersPersInfo = players_info.join(players_salary, on = "Rank", how= "left_outer").join(players_pers, on="Rank", how="left_outer")
playersPersInfo.show(5)

+----+-------------+--------+--------------------+------+-----------+---------------+------------+-------------+-------------+-------------+--------------------+-----+
|Rank|         Name|   Sport|      Wikipedia Page|Gender|  Total Pay|Salary/Winnings|Endorsements|      Country|birthday year|Date of Birth|              Region|goles|
+----+-------------+--------+--------------------+------+-----------+---------------+------------+-------------+-------------+-------------+--------------------+-----+
|  55|Aaron Rodgers|Football|http://en.wikiped...|  Male|$22,000,000|    $11,000,000| $11,000,000|United States|         1981|   1981-09-26|Saginaw, Michigan...|  175|
|  55|Aaron Rodgers|Football|http://en.wikiped...|  Male|$22,000,000|    $11,000,000| $11,000,000|United States|         1983|   1983-12-02|Chico, California...|  188|
|  55|Aaron Rodgers|Football|http://en.wikiped...|  Male|$22,000,000|    $14,500,000|  $7,500,000|United States|         1981|   1981-09-26|Saginaw, Michigan...

### Muestra el tipo de dato de cada columna.

In [47]:
playersPersInfo.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Wikipedia Page: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Total Pay: string (nullable = true)
 |-- Salary/Winnings: string (nullable = true)
 |-- Endorsements: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- birthday year: integer (nullable = true)
 |-- Date of Birth: date (nullable = true)
 |-- Region: string (nullable = true)
 |-- goles: integer (nullable = true)



### Filtra solo por los jugadores que no viven en los Estados Unidos

In [48]:
df = playersPersInfo.select("Rank", "Name" , "Country" ,"Region", "Date of Birth")
df.filter(playersPersInfo["Country"] == "United States").show(5)

+----+---------------+-------------+--------------------+-------------+
|Rank|           Name|      Country|              Region|Date of Birth|
+----+---------------+-------------+--------------------+-------------+
|  55|  Aaron Rodgers|United States|Saginaw, Michigan...|   1981-09-26|
|  55|  Aaron Rodgers|United States|Chico, California...|   1983-12-02|
|  55|  Aaron Rodgers|United States|Saginaw, Michigan...|   1981-09-26|
|  55|  Aaron Rodgers|United States|Chico, California...|   1983-12-02|
|  60|Adrian Gonzalez|United States|San Diego, Califo...|   1982-05-08|
+----+---------------+-------------+--------------------+-------------+
only showing top 5 rows



### Ordena los jugadores del más joven al más veterano

In [49]:
df.sort(desc("Date of Birth")).show(10)

+----+--------------+-------+--------------------+-------------+
|Rank|          Name|Country|              Region|Date of Birth|
+----+--------------+-------+--------------------+-------------+
|  16|        Neymar| Brazil|Mogi das Cruzes, ...|   1992-02-05|
|  66|   Eli Manning| Mexico|Guadalajara, Jali...|   1990-07-18|
|  66|    Drew Brees| Mexico|Guadalajara, Jali...|   1990-07-18|
|  66|Canelo Alvarez| Mexico|Guadalajara, Jali...|   1990-07-18|
|  66|    Drew Brees| Mexico|Guadalajara, Jali...|   1990-07-18|
|  66|   Eli Manning| Mexico|Guadalajara, Jali...|   1990-07-18|
|  66|   Eli Manning| Mexico|Guadalajara, Jali...|   1990-07-18|
|  66|Canelo Alvarez| Mexico|Guadalajara, Jali...|   1990-07-18|
|  66|Canelo Alvarez| Mexico|Guadalajara, Jali...|   1990-07-18|
|  66|    Drew Brees| Mexico|Guadalajara, Jali...|   1990-07-18|
+----+--------------+-------+--------------------+-------------+
only showing top 10 rows



### Escribe el resultado en un nuevo CSV delimitado por pipes “|” y nómbralo csv_result_players_info.csv

In [51]:
playersPersInfo.write.csv("data/csv_result_players_info.csv", sep="|", header=True, mode="overwrite")